In [123]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import math

import pymorphy2
import re
from collections import defaultdict

from tqdm import tqdm, tqdm_notebook
import bisect
from collections import namedtuple,defaultdict
import numpy as np

DocEntry = namedtuple('DocEntry',['doc_id','positions'])

In [124]:
#data
d = ["cat dog cat home cat animal", "duck cow home cow", "dog hunter duck duck dog gun river", "crocodile tiger is cat animal ", "animal crocodile crocodile river"]

In [125]:
x = np.array(d)

In [139]:
""" инвертированный индекс 
    используется для построения матриц вероятностей 
    """
analyzer = pymorphy2.MorphAnalyzer()

def parse_text(text):
    words = (word for word in re.split('\W+', text) if len(word) >0)
    lexems = (analyzer.normal_forms(word)[0] for word in words)
    
    return list(lexems)


class InvertedIndex:
    
    def __init__(self):
        self.dict = defaultdict(list)
        self.texts = dict()
        
    def add_document(self, text, doc_id):
        self.texts[doc_id] = text
        words = parse_text(text)
        
        
        word_to_entry = defaultdict(lambda: [])
        
        for pos, word in enumerate(words):
            
            doc_entry = word_to_entry[word]
            doc_entry.append(pos)
            
        for word, positions in word_to_entry.items():
            postings = self.dict[word]
            entry = DocEntry(doc_id, positions)
            
            i = bisect.bisect_left(postings, entry)
        
            postings.insert(i, entry)
            
    def get_postings(self, word):
        return self.dict[word]
    
    
"""Создание инвертированного индекса"""   
index = InvertedIndex()

for i, line in enumerate(x):
        doc_id = i
        text = line
        index.add_document(text,doc_id)
        
"""Подсчет векторного произведения"""        
def mult_single(x,y):
    #conditions x>0 y>o
    return x*y

def mult_vect(x,y):
    mult = np.vectorize(mult_single)
    return np.sum(mult(x,y))

"""Dkl"""

def dkl_single(x,y):
    if x>0 and y>0:
     
        return x*math.log2(x/y)
    elif x==0: return 0.0
    else: return math.inf

def dkl(x,y):
    sum_dkl = np.vectorize(dkl_single)
    
    return np.sum(sum_dkl(x,y))

def div_matr(D, n, m,py_x,py_t) :
    for i in range(len(x)):
        for j in range(m):
            Div[i,j] = dkl(py_x[:,i],py_t.transpose()[:,j]) 
            
    return Div
    
"""Djs"""
def djs(x,y):
    return 0.5 * dkl(x, (x+y)/2) + 0.5* dkl(y, (x+y)/2)

def djs_vect(Djs,m,pt_x,pt_x_new ):
    for i in range(m):
        
        Djs[i]= djs(pt_x[i,:], pt_x_new[i,:])
    return Djs

"""P(t)"""
def f_pt(pt,m,px,pt_x):
    for i in range(m):
        pt[i] = mult_vect(px, pt_x[i,:])
    return pt

"""P(y|t)"""
def f_py_t(py_t, m,n,pt_x,pxy,pt):
    for i in range(m):
        for j in range(n):
            py_t[i,j] = mult_vect(pt_x[i,:],pxy[:,j])/pt[i]
            
    return py_t
"""Выводит список из  n элементов, соответсвующих  """
def main_words(n, r):
    l = []

    for i in range(n):
        l.append(r.argmax())
        r[r.argmax()] = 0
    return l

#словарь слов: ключ - номер индекса , значение -словао
word_dict = {}
for i, word in enumerate(index.dict.keys()):
    word_dict[i] = word
    
# создает список из n самых важных слов для кластера   
def main_words(n, r):
    l = []

    for i in range(n):
        l.append(r.argmax())
        r[r.argmax()] = 0
    return l

#печатает самых важные слова для кластера
#cl_num - number of cluster
def show_main_words(cl_num):
    for w in clasters_main_words.get(cl_num):
        print("Самые важные слова кластера №" + str(cl_num) + ": " + str(word_dict.get(w)))
        
#show all numbers of docs from  cl,   
def show_text_docs(cl_num):
    show_main_words(cl_num)
    print("\n")
    docs = doc_clast[cl_num] # список документов кластера №cl_num
    for num in docs:
        print("№ документа: " + str(num) + " ТЕКСТ "+ str(x[num]))
       

In [133]:
"""INPUT"""
#beta
beta = 1.2

#число кластеров
m = 2

#covergence parameter 

e = 0.1

"""общее число слов в коллекции"""
summa = 0
for i in x:
    #print(len(word_tokenize(i)))
    summa+=len(word_tokenize(i))

"""P(x,y)"""    
pxy = np.zeros((len(x), len(index.dict.keys())))
for i, word in enumerate(index.dict.keys()):
    a = index.dict.get(word)
    for doc_id , postings in a:
        pxy[doc_id,i] = len(postings)/summa
        
        
"""INITIALIZaTION""" 

"""P(t|x)"""
pt_x = np.random.rand(m, len(x))
    
"""задание вероятности P(x)"""    
px  = np.zeros((x.size))

for n, i in enumerate(x):
    b = len(word_tokenize(i))
    #print(i, b, n)
    px[n] = b/summa
    
"""random P(t|x)"""
pt_x = np.random.rand(m, len(x))

"""P(t)"""
pt = np.zeros(m)
for i in range(m):
    pt[i] = mult_vect(px,pt_x[i,:])
    
"""P(y|t)"""
py_t = np.zeros((m,len(index.dict.keys())) )

for i in range(m):
    for j in range(len(index.dict.keys())):
        py_t[i,j] = mult_vect(pt_x[i,:],pxy[:,j])/pt[i]
        

"""P(y|x)"""
py_x = np.zeros((len(index.dict.keys()),len(x)))
word_index=[]
for i, word in enumerate(index.dict.keys()):
    a = index.dict.get(word)
    word_index.append((i,word))
    for doc_id , postings in a:
        py_x[i,doc_id] = len(postings)/len(index.texts[doc_id].split())
        


In [134]:
        """LOOP"""

Div = np.zeros((len(x), m)) #init div   
Div = div_matr(Div, len(x),m,py_x,py_t)

#new P(t|x)
pt_x_new =pt * np.exp(-beta * Div)
pt_x_new = pt_x_new.transpose()

#init Djs
Djs = np.zeros(m)
Djs = djs_vect(Djs,m,pt_x,pt_x_new )

k = 0

while np.sum(Djs)/len(Djs)>=e:
    #переопрделяем 
    pt_x = pt_x_new    
    #P(t)    
    pt = f_pt(pt,m,px,pt_x)
    
    #P(y|t)
    py_t = f_py_t(py_t, m,len(index.dict.keys()),pt_x,pxy,pt) 
                  
    #P(t|x) new
    pt_x_new = pt * np.exp(-beta * div_matr(Div, len(x),m,py_x,py_t)) 
    pt_x_new = pt_x_new.transpose()
       
    Djs = djs_vect(Djs,m,pt_x,pt_x_new )
    k+=1



In [129]:
pt_x_new

array([[ 0.03095168,  0.00053149,  0.00572824,  0.04532262,  0.01729208],
       [ 0.01769143,  0.0007848 ,  0.0014737 ,  0.04993393,  0.02036563]])

In [130]:
# вектор с номерами кластеров, к которому документ предложит( вероятность больше всего)
vec = np.argmax(pt_x_new, axis=0)

#словарь: ключ - номер кластера, значение - номер документа
doc_clast = defaultdict(list)
for i,value in enumerate(vec):
        doc_clast[value].append(i)

        
#СЛОВАРЬ: ключ - номер кластера, значение - список и n самых важных слов

clasters_main_words = {}
n = 2
py_t_copy = py_t.copy()
for i, row in enumerate(py_t_copy):  
    clasters_main_words[i] = main_words(n, row) 

In [140]:
show_text_docs(0)

Самые важные слова кластера №0: cat
Самые важные слова кластера №0: animal


№ документа: 0 ТЕКСТ cat dog cat home cat animal
№ документа: 2 ТЕКСТ dog hunter duck duck dog gun river


In [136]:
show_text_docs(1)

Самые важные слова кластера №1: crocodile
Самые важные слова кластера №1: cat
№ документа: 1 ТЕКСТ duck cow home cow
№ документа: 3 ТЕКСТ crocodile tiger is cat animal 
№ документа: 4 ТЕКСТ animal crocodile crocodile river
